In [1]:
import numpy as np
import scipy.io as sio
import os
import glob
from scipy import sparse
import random as rd
from keras.models import Sequential 
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
import keras
import pandas as pd
import csv
from random import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sklearn.preprocessing as prep

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import sklearn.metrics as metrics
import csv
import datetime
min_max_scaler = preprocessing.MinMaxScaler()

Using TensorFlow backend.


## constant variable

In [12]:
datasetPath = '/media/hoangnh/TaiLieu/xulianh/NhanDienHatThoc/Dataset/final_feature_data/'
groupDatasetName =['dataset-G1.mat', 'dataset-G2.mat','dataset-G3.mat','dataset-G4.mat']
# groupDatasetName =['dataset-G1.mat']
nGroup = len (groupDatasetName)
performancePath = './Result/performances/'
pcaMatrixPath = '/media/hoangnh/TaiLieu/xulianh/NhanDienHatThoc/Dataset/Model/PCAAll.mat'
nTime = 5
nNegTrainSample = 16 #number of negative sample in each species when training
nPosTrainSample = 80 #number of positive sample in each species when training
nPosValSample = 16
nValSample = 16
nNegValSample = 16
nTrainSample = 80
nFeature = 262
nSpecFeature = 256
nSpatialFeatue = 6
nSpecies = 6
nPca = 50
max_features = nPca + nSpatialFeatue

## normalize feature vector

In [3]:
#------------normarlize spatial data---------------
def normalize(trainSet, valSet):
    allSpatialData= np.asarray([])
    for i in range(0,nSpecies):
        if allSpatialData.shape[0] ==0:
            allSpatialData = trainSet[i][:, nSpecFeature:nFeature]
            allSpatialData = np.concatenate((allSpatialData, valSet[i][:, nSpecFeature:nFeature]), axis = 0)
        else:
            allSpatialData = np.concatenate((allSpatialData, trainSet[i][:, nSpecFeature:nFeature]), axis = 0)
            allSpatialData = np.concatenate((allSpatialData, valSet[i][:, nSpecFeature:nFeature]), axis = 0)
    allSpatialData = min_max_scaler.fit_transform(allSpatialData)

    print allSpatialData.shape

    for i in range(0,nSpecies):
        trainSet[i][:, nSpecFeature:nFeature] = allSpatialData[i*(nPosTrainSample + nPosValSample):i*(nPosTrainSample + nPosValSample) +nPosTrainSample,:]
        valSet[i][:, nSpecFeature:nFeature] = allSpatialData[i*(nPosTrainSample + nPosValSample) + nPosTrainSample:(i+1)*(nPosTrainSample + nPosValSample),:]
    return trainSet,valSet

## defind classifiers

## PCA transformation

In [4]:
# use pca matrix to transfrom spec feature to low-dimenson vector
# firstly, separate spec and spatial feature
# than use pca matrix to transform spec feature
# finally combine transformed spec feature and original spatial feature to unique vector feature.

def pcaTransform(data, n_features):
    pcaContent = sio.loadmat(pcaMatrixPath)
    pcaMatrix =  np.matrix(pcaContent["prinCompMat"])
    specData = data[:, 0:nSpecFeature]
    specMatrix = np.matrix(specData)
    spatialData = data[:, nSpecFeature:nFeature]
    trainsformedSpecData = specData * pcaMatrix[:,0:n_features]
    data = np.concatenate((trainsformedSpecData, spatialData), axis = 1)
    return data

In [5]:
names = ["Random Forest", "Neural Net", "AdaBoost"]

classifiers = [
    RandomForestClassifier(max_depth=10, n_estimators=500, max_features = max_features),
    MLPClassifier(alpha=1),
    AdaBoostClassifier()]

## Define classifiers

In [6]:
RFC = RandomForestClassifier(max_depth=10, n_estimators=500, max_features = max_features)
NNC = MLPClassifier(alpha=1)
ABC = AdaBoostClassifier(n_estimators=500)

## check performance functon

In [7]:
def checkPerformance(clf, trainData, trainLabels, valData, valLabels):
    clf.fit(trainData, trainLabels)
    pridictLabels = clf.predict(valData)
    accuracy = clf.score(valData, valLabels)
    precision = metrics.precision_score(allValLabels, pridictLabels, pos_label = 1)
    recall = metrics.recall_score(allValLabels, pridictLabels, pos_label = 1)
    confusionMatrix = metrics.confusion_matrix(y_true=allValLabels, y_pred=pridictLabels)
    specificity = confusionMatrix[1][1]*1.0/(confusionMatrix[1][0] + confusionMatrix[1][1])
    return np.asarray([recall, precision, accuracy, specificity])

## write performance to csv file function

In [14]:
def writeToCsvFile(fileNames, data):
    temp = []
    for i in range(0, nGroup):
        g = data[i]
        temp.append(g.reshape(nSpecies*nTime, 3).T)
    temp = np.asarray(temp)
    data = temp.reshape(nGroup * 3, nSpecies*nTime)
    with open(fileNames, 'wb') as dictFile:
        writer = csv.writer(dictFile)
        writer.writerow(groupDatasetName)
        writer.writerows(data.T)

In [35]:
g1Names = ['NDC1', 'NV1','NepCT','NepBH', 'NTHY','NDSLH']
g2Names = ['BC15', 'KC111', 'NBK', 'NBP', 'NPT1','TB13']
g3Names = ['CL61', 'PD211', 'R068', 'SHPT1', 'SVN1', 'NBP']
g4Names = ['NT16', 'NT16', 'R068', 'SHPT1', 'SVN1', 'NBP']
speciesNames = []
speciesNames= np.asarray([g1Names, g2Names, g3Names, g4Names])
print speciesNames

[['NDC1' 'NV1' 'NepCT' 'NepBH' 'NTHY' 'NDSLH']
 ['BC15' 'KC111' 'NBK' 'NBP' 'NPT1' 'TB13']
 ['CL61' 'PD211' 'R068' 'SHPT1' 'SVN1' 'NBP']
 ['NT16' 'NT16' 'R068' 'SHPT1' 'SVN1' 'NBP']]


In [36]:
title = ''
group = 0
for partSize in range(2, 6):
    nPart = nSpecies/partSize
    sample = range(0, nSpecies)
    shuffle(sample)
    partArray =[]
    nameArray = []
    for part in range(0,nPart):
        partArray.append(sample[part*partSize: (part + 1)* partSize])
        nameArray.append(speciesNames[group][sample[part*partSize: (part + 1)* partSize]])
    print partArray
    print nameArray

#     speciesNameArray = 
# for group in range(3, 4):
#     dataFileName = groupDatasetName[group]
#     print 'group ' + dataFileName
#     content = sio.loadmat(datasetPath + dataFileName)
#     species = np.asarray(content['dataset']['species'][0][0][0])
#     print species

[[4, 0], [5, 3], [1, 2]]
[array(['NTHY', 'NDC1'],
      dtype='|S5'), array(['NDSLH', 'NepBH'],
      dtype='|S5'), array(['NV1', 'NepCT'],
      dtype='|S5')]
[[1, 3, 0], [4, 5, 2]]
[array(['NV1', 'NepBH', 'NDC1'],
      dtype='|S5'), array(['NTHY', 'NDSLH', 'NepCT'],
      dtype='|S5')]
[[5, 3, 2, 4]]
[array(['NDSLH', 'NepBH', 'NepCT', 'NTHY'],
      dtype='|S5')]
[[1, 2, 0, 3, 4]]
[array(['NV1', 'NepCT', 'NDC1', 'NepBH', 'NTHY'],
      dtype='|S5')]


# check performance based on classifier

In [44]:
groupBasedRecalls = []
groupBasedPrecisions = []
groupBasedAccuracies = []
groupBasedSpecificities = []
for group in range(0, nGroup):
    dataFileName = groupDatasetName[group]
    print 'group ' + dataFileName
    content = sio.loadmat(datasetPath + dataFileName)
    trainSet = np.asarray(content['dataset']['train'][0][0][0])
    print 'train shape', trainSet.shape, 'train feature shape', trainSet[0].shape
    valSet = np.asarray(content['dataset']['valid'])[0][0][0]
    print 'val shape', valSet.shape
    species = np.asarray(content['dataset']['species'])[0][0][0]
    trainSet, valSet = normalize(trainSet, valSet)
#     nSpecies = species.shape[0]

    speciesBasedRecalls = []
    speciesBasedPrecisions = []
    speciesBasedAccuracies = []
    speciesBasedSpecificities = []
    for i in range(0, nSpecies):
        print '++++++++++', str(i+1), 'th species' '++++++++++++++'
#       generate training pos sample
        posSamples = trainSet[i]
        nPosSample = posSamples.shape[0]
        posLabels = np.ones(nPosSample, dtype=np.int)
        negLabels = np.zeros(nNegTrainSample*(nSpecies-1), dtype=np.int)
        
#       generate validate pos sample
        valSamples = valSet[i]
        posValSamples = valSet[i]
        posValLabels = np.ones(nPosValSample, dtype=np.int)
        negValLabels = np.zeros(nNegValSample*(nSpecies-1), dtype=np.int)

        timeBasedRecalls = np.asarray([])
        timeBasedPrecisions = np.asarray([])
        timeBasedAccuracies = np.asarray([])
        timeBasedSpecificities = np.asarray([])
        for time in range(0, nTime):
            print '--------time', time, '---------'
            negTempSamples = []
            #get train negagtive sample in the rest species
            for restSpecies in range(0, i) + range(i+1, nSpecies):
                randSample = range(0, nTrainSample)
                shuffle(randSample)
                negTempSamples.append(trainSet[restSpecies][randSample[0:nNegTrainSample], :])
            negSamples = np.asarray(negTempSamples)
            negSamples = negSamples.reshape((nSpecies-1) * nNegTrainSample, nFeature)
            
            allTrainData = np.concatenate((posSamples, negSamples), axis = 0)
            allTrainLabels = np.concatenate((posLabels, negLabels), axis = 0)
            
            #         generate validate neg samples
            negTempValSamples= []
            for restSpecies in range(0, i) + range(i+1, nSpecies):
                randSample = range(0, nValSample)
                shuffle(randSample)
                negTempValSamples.append(valSet[restSpecies][randSample[0:nNegValSample], :])
            
            negValSamples = np.asarray(negTempValSamples)
            negValSamples=negValSamples.reshape((nSpecies-1)*nNegValSample, nFeature)
            allValData = np.concatenate((posValSamples, negValSamples), axis = 0)
            allValLabels = np.concatenate((posValLabels, negValLabels), axis = 0)

            
            
            transformedTrainData = pcaTransform(allTrainData, n_features= nPca)
            transformedValData = pcaTransform(allValData, n_features= nPca)
            
#             print 'allTrainData shape', allTrainData.shape
            
            rfcPerformance = checkPerformance(RFC, transformedTrainData, allTrainLabels, transformedValData, allValLabels)
            nncPerformance = checkPerformance(NNC, transformedTrainData, allTrainLabels, transformedValData, allValLabels)
            abcPerformance = checkPerformance(ABC, transformedTrainData, allTrainLabels, transformedValData, allValLabels)
            
            recalls = np.asarray([rfcPerformance[0], nncPerformance[0], abcPerformance[0]])
            precisions = np.asarray([rfcPerformance[1], nncPerformance[1], abcPerformance[1]])
            accuracies = np.asarray([rfcPerformance[2], nncPerformance[2], abcPerformance[2]])
            specificities = np.asarray([rfcPerformance[3], nncPerformance[3], abcPerformance[3]])
            print "recalls", recalls
            print "precisions", precisions
            print "accuracies", accuracies
            print "specificities", specificities
            if timeBasedRecalls.shape[0] == 0:
                timeBasedRecalls = recalls
                timeBasedPrecisions = precisions
                timeBasedAccuracies = accuracies
                timeBasedSpecificities = specificities
            else:
                timeBasedRecalls = np.concatenate((timeBasedRecalls, recalls), axis = 0)
                timeBasedPrecisions = np.concatenate((timeBasedPrecisions, precisions), axis = 0)
                timeBasedAccuracies = np.concatenate((timeBasedAccuracies, accuracies), axis = 0)
                timeBasedSpecificities = np.concatenate((timeBasedSpecificities, specificities), axis = 0)
            
        timeBasedRecalls = timeBasedRecalls.reshape(nTime, 3)
        timeBasedPrecisions = timeBasedPrecisions.reshape(nTime, 3)
        timeBasedAccuracies = timeBasedAccuracies.reshape(nTime, 3)
        timeBasedSpecificities = timeBasedSpecificities.reshape(nTime, 3)

        speciesBasedRecalls.append(timeBasedRecalls)
        speciesBasedPrecisions.append(timeBasedPrecisions)
        speciesBasedAccuracies.append(timeBasedAccuracies)
        speciesBasedSpecificities.append(timeBasedAccuracies)
    speciesBasedRecalls = np.asarray(speciesBasedRecalls)
    speciesBasedPrecisions = np.asarray(speciesBasedPrecisions)
    speciesBasedAccuracies = np.asarray(speciesBasedAccuracies)
    speciesBasedSpecificities = np.asarray(speciesBasedSpecificities)
    
    speciesBasedRecalls = speciesBasedRecalls.reshape(nSpecies, nTime, 3)
    speciesBasedPrecisions = speciesBasedPrecisions.reshape(nSpecies, nTime, 3)
    speciesBasedAccuracies = speciesBasedAccuracies.reshape(nSpecies, nTime, 3)
    speciesBasedSpecificities = speciesBasedSpecificities.reshape(nSpecies, nTime, 3)
    
    groupBasedRecalls.append(speciesBasedRecalls)
    groupBasedPrecisions.append(speciesBasedPrecisions)
    groupBasedAccuracies.append(speciesBasedAccuracies)
    groupBasedSpecificities.append(speciesBasedSpecificities)
    
groupBasedRecalls = np.asarray(groupBasedRecalls)
groupBasedPrecisions = np.asarray(groupBasedPrecisions)
groupBasedAccuracies = np.asarray(groupBasedAccuracies)
groupBasedSpecificities = np.asarray(groupBasedSpecificities)

groupBasedRecalls = groupBasedRecalls.reshape(nGroup, nSpecies, nTime, 3)
groupBasedPrecisions= groupBasedPrecisions.reshape(nGroup, nSpecies, nTime, 3)
groupBasedAccuracies = groupBasedAccuracies.reshape(nGroup, nSpecies, nTime, 3)
groupBasedSpecificities = groupBasedSpecificities.reshape(nGroup, nSpecies, nTime, 3)
# save to npy files
currentDT = datetime.datetime.now()
print (str(currentDT))
np.save(performancePath +'precisions' + str(currentDT), groupBasedPrecisions)
np.save(performancePath +'recalls'+ str(currentDT), groupBasedRecalls)
np.save(performancePath +'accuracies'+ str(currentDT), groupBasedAccuracies)
np.save(performancePath +'specifitities'+ str(currentDT), groupBasedSpecificities)

# save to csv files
writeToCsvFile('accuracy'+ str(currentDT) +'.csv', groupBasedAccuracies)
writeToCsvFile('recall'+ str(currentDT) +'.csv', groupBasedRecalls)
writeToCsvFile('precision'+ str(currentDT) +'.csv', groupBasedPrecisions)
writeToCsvFile('specifitities'+ str(currentDT) +'.csv', groupBasedSpecificities)


group dataset-G1.mat
train shape (6,) train feature shape (80, 262)
val shape (6,)
(576, 6)
++++++++++ 1 th species++++++++++++++
--------time 0 ---------
recalls [ 0.9375  0.9375  0.875 ]
precisions [ 0.46875     0.42857143  0.42424242]
accuracies [ 0.8125   0.78125  0.78125]
specificities [ 0.9375  0.9375  0.875 ]
--------time 1 ---------
recalls [ 0.9375  0.9375  0.875 ]
precisions [ 0.51724138  0.44117647  0.4375    ]
accuracies [ 0.84375     0.79166667  0.79166667]
specificities [ 0.9375  0.9375  0.875 ]
--------time 2 ---------
recalls [ 0.9375  0.9375  0.875 ]
precisions [ 0.625       0.44117647  0.48275862]
accuracies [ 0.89583333  0.79166667  0.82291667]
specificities [ 0.9375  0.9375  0.875 ]
--------time 3 ---------
recalls [ 0.875   0.9375  0.875 ]
precisions [ 0.58333333  0.46875     0.4516129 ]
accuracies [ 0.875       0.8125      0.80208333]
specificities [ 0.875   0.9375  0.875 ]
--------time 4 ---------
recalls [ 0.9375  0.9375  0.875 ]
precisions [ 0.48387097  0.48387

## For testing

In [18]:
from sklearn.metrics import confusion_matrix
y_true = [0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
y_pred = [0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
con = confusion_matrix(y_true, y_pred)
precision = metrics.precision_score(y_true, y_pred, pos_label = 0)
recall = metrics.recall_score(y_true, y_pred, pos_label = 0)
print recall
print con
print precision

0.75
[[3 1]
 [4 2]]
0.428571428571


In [167]:
a = np.asarray([1, 2, 3])
b = np.asarray([1, 2, 3])
c = []
c.append(a)
c.append(b)
d = np.asarray(c)

e = np.asarray([[1,2,3],[1,3,3]])

In [130]:
i = 10
nSpecies = 16
for i in range(0, i) + range(i+1, nSpecies):
    randSample = shuffle(range(0, nNegSample))
range(0, nNegSample)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [105]:
content = sio.loadmat(testFile)
# prep.normalize([[1,2,4,2,3], [1,2,4,2,3]])

array([[ 0.17149859,  0.34299717,  0.68599434,  0.34299717,  0.51449576],
       [ 0.17149859,  0.34299717,  0.68599434,  0.34299717,  0.51449576]])

In [199]:
species = np.asarray(content['dataset']['species'])[0][0][0]
print species.shape[0]
# species
G1 = ['NDC1', 'NV1', 'NepCT', 'NepBH', 'NTHY', 'NDSLH']


6


In [117]:
trainSet = np.asarray(content['dataset']['train'][0][0][0])
valSet = np.asarray(content['dataset']['valid'])[0][0][0]
# trainSet[0][1,:]
print trainSet.shape
print valSet.shape
maxValue = 0;
for childSet in trainSet:
    maxValue = max(maxValue, np.amax(childSet))
    print maxValue
print trainSet[0].shape
print maxValue
# np.amax(trainSet)

(80, 262)
(6,)
18619.0
20281.0
20281.0
20476.0
20476.0
20476.0
(80, 262)
20476.0


In [110]:
trainSet, valSet = normalize(trainSet, valSet)

In [112]:

names = ["Random Forest", "Neural Net", "AdaBoost"]

classifiers = [
    RandomForestClassifier(max_depth=10, n_estimators=10, max_features=250),
    MLPClassifier(alpha=1),
    AdaBoostClassifier()]

In [ ]:
confusionMat = zeros(2,2);
for i=1:npointTest:
    confusionMat(allLabel(i),predictLabel(i))=confusionMat(allLabel(i),predictLabel(i))+1;


In [ ]:
print len(allTrainData), len(classifiers) + 1
for idx,(name, clf) in enumerate(zip(names, classifiers)):
    clf.fit(allTrainData, allTrainLabels)
    pridicLabels = clf.predict(allTestData)
    confusionMatrix = metrics.confusion_matrix(pridicLabels, validLabels)
#     Recall(j) = confusionMat(1,1)/(confusionMat(1,1) + confusionMat(1,2))
#     Precision(j) = confusionMat(1,1)/(confusionMat(1,1) + confusionMat(2,1))